# 80 cereals dataset analysis using MySQL and python

## Fields in the dataset:
- brand: Name of cereal
- mfr: Manufacturer of cereal:
1. A = American Home Food Products,
2. G = General Mills,
3. K = Kelloggs,
4. N = Nabisco,
5. P = Post,
6. Q = Quaker Oats,
7. R = Ralston Purina,
- type:
1. cold
2. hot
- calories: calories per serving
- protein: grams of protein
- fat: grams of fat
- sodium: milligrams of sodium
- fiber: grams of dietary fiber
- carbo: grams of complex carbohydrates
- sugars: grams of sugars
- potass: milligrams of potassium
- vitamins: vitamins and minerals - 0, 25, or 100, indicating the typical percentage of FDA recommended
- shelf: display shelf (1, 2, or 3, counting from the floor)
- weight: weight in ounces of one serving
- cups: number of cups in one serving
- rating: a rating of the cereals (Possibly from Consumer Reports)

Dataset was downloaded from [kaggle](https://www.kaggle.com/datasets/crawford/80-cereals).

In [3]:
%pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import mysql.connector
import matplotlib.pyplot as plt
import json

## Connecting with the server

In [3]:
with open('secrets.json') as json_file:
    secrets = json.load(json_file)

In [4]:
mydb = mysql.connector.connect(
    host = secrets["host"],
    user = secrets['user'],
    passwd = secrets['password'],
    database = secrets['database']
)
print(mydb)

In [5]:
mycursor = mydb.cursor()
mycursor.execute('SELECT VERSION ()')
row = mycursor.fetchone()
print('Server Version is',row)

Server Version is ('8.0.34',)


## Selecting main table from cereals database

In [8]:
mycursor = mydb.cursor(buffered=True)
mycursor.execute("SELECT * FROM main;")

myresult = mycursor.fetchall()

for x in myresult:
    print(x)

('100% Bran', 'N', 'C', 70, 4, 1, 130, 10.0, 5.0, 6, 280, 25, 3, 1.0, 0.33, 68.403)
('100% Natural Bran', 'Q', 'C', 120, 3, 5, 15, 2.0, 8.0, 8, 135, 0, 3, 1.0, 1.0, 33.9837)
('All-Bran', 'K', 'C', 70, 4, 1, 260, 9.0, 7.0, 5, 320, 25, 3, 1.0, 0.33, 59.4255)
('All-Bran with Extra Fiber', 'K', 'C', 50, 4, 0, 140, 14.0, 8.0, 0, 330, 25, 3, 1.0, 0.5, 93.7049)
('Almond Delight', 'R', 'C', 110, 2, 2, 200, 1.0, 14.0, 8, -1, 25, 3, 1.0, 0.75, 34.3848)
('Apple Cinnamon Cheerios', 'G', 'C', 110, 2, 2, 180, 1.5, 10.5, 10, 70, 25, 1, 1.0, 0.75, 29.5095)
('Apple Jacks', 'K', 'C', 110, 2, 0, 125, 1.0, 11.0, 14, 30, 25, 2, 1.0, 1.0, 33.1741)
('Basic 4', 'G', 'C', 130, 3, 2, 210, 2.0, 18.0, 8, 100, 25, 3, 1.33, 0.75, 37.0386)
('Bran Chex', 'R', 'C', 90, 2, 1, 200, 4.0, 15.0, 6, 125, 25, 1, 1.0, 0.67, 49.1203)
('Bran Flakes', 'P', 'C', 90, 3, 0, 210, 5.0, 13.0, 5, 190, 25, 3, 1.0, 0.67, 53.3138)
('Cap n Crunch', 'Q', 'C', 120, 1, 2, 220, 0.0, 12.0, 12, 35, 25, 2, 1.0, 0.75, 18.0429)
('Cheerios', 'G', 'C

In [17]:
query = """
    CREATE TABLE IF NOT EXISTS nutrition 
    (brand VARCHAR(255), 
    calories INT, 
    protein INT, 
    fat INT, 
    sodium INT, 
    fiber FLOAT, 
    carbo FLOAT, 
    sugars INT, 
    potass INT, 
    vitamins INT, 
    PRIMARY KEY (brand))
"""
mycursor.execute(query)

In [ ]:
mycursor.execute('SHOW TABLES')
for x in mycursor:
    print(x)

('main',)
('nutrition',)
('store',)


In [26]:
mycursor.execute("SELECT brand, calories, protein, fat, sodium, fiber, carbo, sugars, potass, vitamins FROM main")

# Pobrane dane
result = mycursor.fetchall()

# Wstaw dane do tabeli docelowej
for row in result:
    insert_query = """
    INSERT IGNORE INTO nutrition (
        brand, calories, protein, fat, sodium, 
        fiber, carbo, sugars, potass, vitamins
        ) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
        
    mycursor.execute(insert_query, row)

In [ ]:
mycursor.execute("SELECT * FROM nutrition;")

myresult = mycursor.fetchall()

for x in myresult:
    print(x)

('100% Bran', 70, 4, 1, 130, 10.0, 5.0, 6, 280, 25)
('100% Natural Bran', 120, 3, 5, 15, 2.0, 8.0, 8, 135, 0)
('All-Bran', 70, 4, 1, 260, 9.0, 7.0, 5, 320, 25)
('All-Bran with Extra Fiber', 50, 4, 0, 140, 14.0, 8.0, 0, 330, 25)
('Almond Delight', 110, 2, 2, 200, 1.0, 14.0, 8, -1, 25)
('Apple Cinnamon Cheerios', 110, 2, 2, 180, 1.5, 10.5, 10, 70, 25)
('Apple Jacks', 110, 2, 0, 125, 1.0, 11.0, 14, 30, 25)
('Basic 4', 130, 3, 2, 210, 2.0, 18.0, 8, 100, 25)
('Bran Chex', 90, 2, 1, 200, 4.0, 15.0, 6, 125, 25)
('Bran Flakes', 90, 3, 0, 210, 5.0, 13.0, 5, 190, 25)
('Cap n Crunch', 120, 1, 2, 220, 0.0, 12.0, 12, 35, 25)
('Cheerios', 110, 6, 2, 290, 2.0, 17.0, 1, 105, 25)
('Cinnamon Toast Crunch', 120, 1, 3, 210, 0.0, 13.0, 9, 45, 25)
('Clusters', 110, 3, 2, 140, 2.0, 13.0, 7, 105, 25)
('Cocoa Puffs', 110, 1, 1, 180, 0.0, 12.0, 13, 55, 25)
('Corn Chex', 110, 2, 0, 280, 0.0, 22.0, 3, 25, 25)
('Corn Flakes', 100, 2, 0, 290, 1.0, 21.0, 2, 35, 25)
('Corn Pops', 110, 1, 0, 90, 1.0, 13.0, 12, 20, 25)

In [23]:
query = """
    CREATE TABLE IF NOT EXISTS store (
        brand VARCHAR(255), 
        mfr VARCHAR(1), 
        type VARCHAR(1), 
        shelf INT, 
        weight FLOAT, 
        cups FLOAT, 
        rating FLOAT, 
        PRIMARY KEY (brand)
        )
"""

mycursor.execute(query)

In [27]:
mycursor.execute("SELECT brand, mfr, type, shelf, weight, cups, rating FROM main")

# Pobrane dane
result = mycursor.fetchall()

# Wstaw dane do tabeli docelowej
for row in result:
    insert_query = """
    INSERT IGNORE INTO store (
        brand, mfr, type, shelf, 
        weight, cups, rating
        ) 
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    mycursor.execute(insert_query, row)

In [28]:
mycursor.execute("SELECT * FROM store;")

myresult = mycursor.fetchall()

for x in myresult:
    print(x)

('100% Bran', 'N', 'C', 3, 1.0, 0.33, 68.403)
('100% Natural Bran', 'Q', 'C', 3, 1.0, 1.0, 33.9837)
('All-Bran', 'K', 'C', 3, 1.0, 0.33, 59.4255)
('All-Bran with Extra Fiber', 'K', 'C', 3, 1.0, 0.5, 93.7049)
('Almond Delight', 'R', 'C', 3, 1.0, 0.75, 34.3848)
('Apple Cinnamon Cheerios', 'G', 'C', 1, 1.0, 0.75, 29.5095)
('Apple Jacks', 'K', 'C', 2, 1.0, 1.0, 33.1741)
('Basic 4', 'G', 'C', 3, 1.33, 0.75, 37.0386)
('Bran Chex', 'R', 'C', 1, 1.0, 0.67, 49.1203)
('Bran Flakes', 'P', 'C', 3, 1.0, 0.67, 53.3138)
('Cap n Crunch', 'Q', 'C', 2, 1.0, 0.75, 18.0429)
('Cheerios', 'G', 'C', 1, 1.0, 1.25, 50.765)
('Cinnamon Toast Crunch', 'G', 'C', 2, 1.0, 0.75, 19.8236)
('Clusters', 'G', 'C', 3, 1.0, 0.5, 40.4002)
('Cocoa Puffs', 'G', 'C', 2, 1.0, 1.0, 22.7364)
('Corn Chex', 'R', 'C', 1, 1.0, 1.0, 41.445)
('Corn Flakes', 'K', 'C', 1, 1.0, 1.0, 45.8633)
('Corn Pops', 'K', 'C', 2, 1.0, 1.0, 35.7828)
('Count Chocula', 'G', 'C', 2, 1.0, 1.0, 22.3965)
('Cracklin Oat Bran', 'K', 'C', 3, 1.0, 0.5, 40.4488)

## Average content of each nutrient

In [31]:
query = """
    SELECT
        AVG(calories) AS avg_calories,
        AVG(protein) AS avg_protein,
        AVG(fat) AS avg_fat,
        AVG(sodium) AS avg_sodium,
        AVG(fiber) AS avg_fiber,
        AVG(carbo) AS avg_carbo,
        AVG(sugars) AS avg_sugars,
        AVG(potass) AS avg_potass,
        AVG(vitamins) AS avg_vitamins
    FROM
        nutrition
"""

In [35]:
mycursor.execute(query)

# Fetch the results
results = mycursor.fetchone()

column_names = [desc[0] for desc in mycursor.description]

    # Print the column names and their corresponding average values
for i in range(len(column_names)):
    print(f"{column_names[i]}: {results[i]}")

avg_calories: 106.8831
avg_protein: 2.5455
avg_fat: 1.0130
avg_sodium: 159.6753
avg_fiber: 2.151948052567321
avg_carbo: 14.597402597402597
avg_sugars: 6.9221
avg_potass: 96.0779
avg_vitamins: 28.2468


## Calories per ounce

In [54]:
query = """
    SELECT brand,
           calories / weight AS calories_per_ounce
    FROM main
    ORDER BY calories_per_ounce desc
    """
mycursor.execute(query)

results = mycursor.fetchall()

for row in results:
    brand = row[0]
    weight_per_ounce = row[1]
    print(f"Brand: {brand}, Calories per Ounce: {weight_per_ounce}")

Brand: Muesli Raisins Dates Almonds, Calories per Ounce: 150.0
Brand: Muesli Raisins Peches Pecans, Calories per Ounce: 150.0
Brand: 100% Natural Bran, Calories per Ounce: 120.0
Brand: Great Grains Pecan, Calories per Ounce: 120.0
Brand: Honey Graham Ohs, Calories per Ounce: 120.0
Brand: Cinnamon Toast Crunch, Calories per Ounce: 120.0
Brand: Cap n Crunch, Calories per Ounce: 120.0
Brand: Nut&Honey Crunch, Calories per Ounce: 120.0
Brand: Just Right Crunchy  Nuggets, Calories per Ounce: 110.0
Brand: Frosted Flakes, Calories per Ounce: 110.0
Brand: Fruity Pebbles, Calories per Ounce: 110.0
Brand: Golden Grahams, Calories per Ounce: 110.0
Brand: Grape-Nuts, Calories per Ounce: 110.0
Brand: Wheaties Honey Gold, Calories per Ounce: 110.0
Brand: Trix, Calories per Ounce: 110.0
Brand: Honey Nut Cheerios, Calories per Ounce: 110.0
Brand: Honey-comb, Calories per Ounce: 110.0
Brand: Rice Chex, Calories per Ounce: 110.0
Brand: Kix, Calories per Ounce: 110.0
Brand: Lucky Charms, Calories per Oun

## Manufacturers

### Ratings

In [52]:
query = """
    SELECT mfr, AVG(rating) AS average_rating
    FROM store
    GROUP BY mfr
    ORDER BY average_rating DESC
    """
mycursor.execute(query)

results = mycursor.fetchall()

for row in results:
    manufacturer = row[0]
    average_rating = row[1]
    print(f"Manufacturer: {manufacturer}, Average Rating: {average_rating}")

Manufacturer: N, Average Rating: 67.96856689453125
Manufacturer: A, Average Rating: 54.85089874267578
Manufacturer: K, Average Rating: 44.03846093882685
Manufacturer: Q, Average Rating: 42.91599988937378
Manufacturer: P, Average Rating: 41.70573298136393
Manufacturer: R, Average Rating: 41.54300022125244
Manufacturer: G, Average Rating: 34.4858455657959


### Shelf location

In [56]:
query = """
    SELECT mfr, AVG(shelf) AS average_shelf
    FROM store
    GROUP BY mfr
    ORDER BY average_shelf ASC
    """
mycursor.execute(query)

results = mycursor.fetchall()

for row in results:
    manufacturer = row[0]
    average_rating = row[1]
    print(f"Manufacturer: {manufacturer}, Average Rating: {average_rating}")

Manufacturer: N, Average Rating: 1.6667
Manufacturer: R, Average Rating: 2.0000
Manufacturer: A, Average Rating: 2.0000
Manufacturer: G, Average Rating: 2.1364
Manufacturer: K, Average Rating: 2.3478
Manufacturer: Q, Average Rating: 2.3750
Manufacturer: P, Average Rating: 2.4444


### Nutrition

In [7]:
query = """
SELECT brand, mfr,  protein, fat, sodium, fiber, carbo, sugars, potass, vitamins
FROM main
"""

# Pobieranie danych z bazy danych i tworzenie DataFrame
df = pd.read_sql(query, con=mydb)
df.head(10)

/tmp/ipykernel_36452/2087260085.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=mydb)


,brand,mfr,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins
0,100% Bran,N,4,1,130,10.0,5.0,6,280,25
1,100% Natural Bran,Q,3,5,15,2.0,8.0,8,135,0
2,All-Bran,K,4,1,260,9.0,7.0,5,320,25
3,All-Bran with Extra Fiber,K,4,0,140,14.0,8.0,0,330,25
4,Almond Delight,R,2,2,200,1.0,14.0,8,-1,25
5,Apple Cinnamon Cheerios,G,2,2,180,1.5,10.5,10,70,25
6,Apple Jacks,K,2,0,125,1.0,11.0,14,30,25
7,Basic 4,G,3,2,210,2.0,18.0,8,100,25
8,Bran Chex,R,2,1,200,4.0,15.0,6,125,25
9,Bran Flakes,P,3,0,210,5.0,13.0,5,190,25


In [8]:
def normalize_column(column):
    max_value = column.max()
    normalized_column = column / max_value
    return normalized_column

In [9]:
columns_to_normalize = df.columns[2:]

for column_name in columns_to_normalize:
    df[column_name] = normalize_column(df[column_name])
df.head(5)

,brand,mfr,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins
0,100% Bran,N,0.666667,0.2,0.406250,0.714286,0.217391,0.400000,0.848485,0.25
1,100% Natural Bran,Q,0.500000,1.0,0.046875,0.142857,0.347826,0.533333,0.409091,0.00
2,All-Bran,K,0.666667,0.2,0.812500,0.642857,0.304348,0.333333,0.969697,0.25
3,All-Bran with Extra Fiber,K,0.666667,0.0,0.437500,1.000000,0.347826,0.000000,1.000000,0.25
4,Almond Delight,R,0.333333,0.4,0.625000,0.071429,0.608696,0.533333,-0.003030,0.25


In [11]:
sums = df.iloc[:, 2:].sum(axis=1)

df['total_sum'] = sums
sorted_df = df.sort_values(by='total_sum', ascending=False)

print(sorted_df[['mfr', 'total_sum']])

   mfr  total_sum
70   G   4.861941
52   P   4.203044
2    K   4.179402
45   R   4.143839
39   K   4.131551
..  ..        ...
20   N   1.731442
57   Q   1.649379
63   N   1.531150
55   Q   0.991060
54   Q   0.777339

[77 rows x 2 columns]


In [16]:
average_total_sum_by_mfr = df.groupby('mfr')['total_sum'].mean()
mfr_nutrition = average_total_sum_by_mfr.sort_values(ascending = False)
# Wyświetlanie wyników
print(mfr_nutrition)

mfr
G    3.157577
K    3.127888
R    3.114828
P    2.995377
Q    2.307260
A    2.300198
N    2.175322
Name: total_sum, dtype: float64


## Healthiest choice

In [60]:
query = """
SELECT brand, protein, fat, sodium, fiber, carbo, sugars, potass, vitamins
FROM nutrition
"""

# Pobieranie danych z bazy danych i tworzenie DataFrame
df = pd.read_sql(query, con=mydb)
df.head(10)

/tmp/ipykernel_4230/3678663200.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=mydb)


,brand,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins
0,100% Bran,4,1,130,10.0,5.0,6,280,25
1,100% Natural Bran,3,5,15,2.0,8.0,8,135,0
2,All-Bran,4,1,260,9.0,7.0,5,320,25
3,All-Bran with Extra Fiber,4,0,140,14.0,8.0,0,330,25
4,Almond Delight,2,2,200,1.0,14.0,8,-1,25
5,Apple Cinnamon Cheerios,2,2,180,1.5,10.5,10,70,25
6,Apple Jacks,2,0,125,1.0,11.0,14,30,25
7,Basic 4,3,2,210,2.0,18.0,8,100,25
8,Bran Chex,2,1,200,4.0,15.0,6,125,25
9,Bran Flakes,3,0,210,5.0,13.0,5,190,25


In [77]:
columns_to_normalize = df.columns[1:]

for column_name in columns_to_normalize:
    df[column_name] = normalize_column(df[column_name])
df.head(5)

,brand,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,total_sum
0,100% Bran,0.666667,0.2,0.406250,0.714286,0.217391,0.400000,0.848485,0.25,0.761646
1,100% Natural Bran,0.500000,1.0,0.046875,0.142857,0.347826,0.533333,0.409091,0.00,0.612920
2,All-Bran,0.666667,0.2,0.812500,0.642857,0.304348,0.333333,0.969697,0.25,0.859616
3,All-Bran with Extra Fiber,0.666667,0.0,0.437500,1.000000,0.347826,0.000000,1.000000,0.25,0.761423
4,Almond Delight,0.333333,0.4,0.625000,0.071429,0.608696,0.533333,-0.003030,0.25,0.579760


In [80]:
sums = df.iloc[:, 1:].sum(axis=1)

df['total_sum'] = sums
sorted_df = df.sort_values(by='total_sum', ascending=False)

print(sorted_df[['brand', 'total_sum']])
print(f'The healthiest choice is {sorted_df.iloc[0, 0]}.')

                           brand  total_sum
70             Total Raisin Bran  15.585824
52         Post Nat. Raisin Bran  13.473612
2                       All-Bran  13.397822
45  Muesli Raisins Peches Pecans  13.283820
39        Just Right Fruit & Nut  13.244427
..                           ...        ...
20        Cream of Wheat (Quick)   5.550447
57                Quaker Oatmeal   5.287380
63                Shredded Wheat   4.908376
55                  Puffed Wheat   3.177019
54                   Puffed Rice   2.491898

[77 rows x 2 columns]
The healthiest choice is Total Raisin Bran.
